# VenueSignal: Complete MLOps Pipeline
### AAI-540 Group 6 - Yelp Business Rating Prediction

---

## Project Overview

This notebook implements a complete end-to-end MLOps pipeline for predicting Yelp business ratings with a focus on parking availability constraints. The pipeline demonstrates MLOps best practices including:

- **Data Lake Management**: S3-based data storage with proper versioning
- **Data Cataloging**: Athena tables for queryable data access
- **Feature Engineering**: Scalable feature store implementation
- **Model Development**: Baseline and advanced models with proper evaluation
- **Model Deployment**: SageMaker endpoints for inference
- **Monitoring**: Comprehensive model, data, and infrastructure monitoring

**Key Feature**: Uses AWS Account ID for bucket naming to enable each team member to run independently in their own AWS Learning Lab environment.

---

## Table of Contents

1. [Setup & Configuration](#section-1)
2. [Data Lake Setup](#section-2)
3. [Athena Tables & Data Cataloging](#section-3)
4. [Exploratory Data Analysis](#section-4)
5. [Feature Engineering & Feature Store](#section-5)
6. [Model Training](#section-6)
   - 6.1 Benchmark Models
   - 6.2 XGBoost Model
   - 6.3 Model Comparison
7. [Model Deployment](#section-7)
8. [Monitoring & Observability](#section-8)

---

## 1. Setup & Configuration <a id='section-1'></a>

This section:
- Verifies Python version
- Imports all required libraries
- Retrieves AWS Account ID for unique resource naming
- Initializes AWS clients and SageMaker session
- Configures S3 buckets using Account ID pattern

In [ ]:
# Verify Python version
!python --version

### 1.1 Import Required Libraries

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import os
import json
import re
import time
from collections import Counter
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# AWS SDK
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from botocore.exceptions import ClientError

# SageMaker
import sagemaker
from sagemaker import get_execution_role
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Athena
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

# Model training and evaluation
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)
from sklearn.linear_model import LinearRegression

# Google Drive download
import gdown

print("✅ All libraries imported successfully")

### 1.2 Retrieve AWS Account ID

**IMPORTANT**: This retrieves your unique AWS Account ID which will be used to create unique S3 bucket names.
This allows each team member to run this notebook independently in their own AWS Learning Lab environment.

In [ ]:
try:
    # Get AWS Account ID
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    print(f"✅ Successfully retrieved AWS Account ID: {account_id}")
except Exception as e:
    print(f"❌ Cannot retrieve account information: {e}")
    raise

### 1.3 Initialize AWS Clients and SageMaker Session

In [ ]:
# AWS Region
REGION = "us-east-1"

# Initialize SageMaker session
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Initialize AWS clients
s3_client = boto3.client("s3", region_name=REGION)
s3_resource = boto3.resource("s3", region_name=REGION)
athena_client = boto3.client("athena", region_name=REGION)
sagemaker_client = boto3.client("sagemaker", region_name=REGION)
cloudwatch_client = boto3.client("cloudwatch", region_name=REGION)
logs_client = boto3.client("logs", region_name=REGION)

print(f"✅ AWS Region: {REGION}")
print(f"✅ SageMaker Execution Role: {role}")
print(f"✅ AWS clients initialized successfully")

### 1.4 Configure S3 Buckets with Account ID Pattern

**IMPORTANT**: All S3 buckets are created with your Account ID to ensure uniqueness.
This pattern is used throughout the entire pipeline.

In [ ]:
# Base bucket name (shared across team)
BASE_BUCKET_NAME = "yelp-aai540-group6"

# Individual buckets with Account ID for each team member
DATA_BUCKET = f"{BASE_BUCKET_NAME}-{account_id}"  # Raw data storage
ATHENA_BUCKET = f"{BASE_BUCKET_NAME}-athena-{account_id}"  # Athena queries and results
FEATURE_BUCKET = f"{BASE_BUCKET_NAME}-features-{account_id}"  # Feature store offline
MODEL_BUCKET = f"{BASE_BUCKET_NAME}-models-{account_id}"  # Model artifacts
MONITORING_BUCKET = f"{BASE_BUCKET_NAME}-monitoring-{account_id}"  # Monitoring data

# S3 Prefixes (paths within buckets)
RAW_DATA_PREFIX = "yelp-dataset/json/"
PARQUET_PREFIX = "yelp-dataset/parquet/"
ATHENA_RESULTS_PREFIX = "athena-results/"
FEATURE_PREFIX = "feature-store/"
MODEL_PREFIX = "models/"
MONITORING_PREFIX = "monitoring/"

# Full S3 paths
ATHENA_RESULTS_S3 = f"s3://{ATHENA_BUCKET}/{ATHENA_RESULTS_PREFIX}"

# Athena Database
ATHENA_DB = "yelp"

# Store configuration
%store REGION
%store account_id
%store DATA_BUCKET
%store ATHENA_BUCKET
%store FEATURE_BUCKET
%store MODEL_BUCKET
%store MONITORING_BUCKET
%store ATHENA_RESULTS_S3
%store ATHENA_DB

# Display configuration
print("="*80)
print("S3 BUCKET CONFIGURATION (Account-Specific)")
print("="*80)
print(f"AWS Account ID:     {account_id}")
print(f"AWS Region:         {REGION}")
print()
print("S3 Buckets:")
print(f"  Data Bucket:      {DATA_BUCKET}")
print(f"  Athena Bucket:    {ATHENA_BUCKET}")
print(f"  Feature Bucket:   {FEATURE_BUCKET}")
print(f"  Model Bucket:     {MODEL_BUCKET}")
print(f"  Monitoring:       {MONITORING_BUCKET}")
print()
print("Athena Configuration:")
print(f"  Database:         {ATHENA_DB}")
print(f"  Results Location: {ATHENA_RESULTS_S3}")
print("="*80)

### 1.5 Create S3 Buckets

This creates all required S3 buckets for the pipeline. Each bucket is unique to your AWS account.

In [ ]:
def create_bucket_if_not_exists(bucket_name, region=REGION):
    """
    Create an S3 bucket if it doesn't already exist.
    
    Args:
        bucket_name: Name of the bucket to create
        region: AWS region for the bucket
    
    Returns:
        True if bucket was created or already exists, False otherwise
    """
    try:
        # Check if bucket exists
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"  ✅ Bucket already exists: {bucket_name}")
        return True
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == '404':
            # Bucket doesn't exist, create it
            try:
                if region == 'us-east-1':
                    s3_client.create_bucket(Bucket=bucket_name)
                else:
                    s3_client.create_bucket(
                        Bucket=bucket_name,
                        CreateBucketConfiguration={'LocationConstraint': region}
                    )
                print(f"  ✅ Created bucket: {bucket_name}")
                return True
            except ClientError as create_error:
                print(f"  ❌ Error creating bucket {bucket_name}: {create_error}")
                return False
        else:
            print(f"  ❌ Error checking bucket {bucket_name}: {e}")
            return False

# Create all required buckets
print("Creating S3 buckets...")
buckets = [
    DATA_BUCKET,
    ATHENA_BUCKET,
    FEATURE_BUCKET,
    MODEL_BUCKET,
    MONITORING_BUCKET
]

all_success = True
for bucket in buckets:
    if not create_bucket_if_not_exists(bucket):
        all_success = False

if all_success:
    print("\n✅ All S3 buckets are ready!")
else:
    print("\n⚠️ Some buckets could not be created. Please check errors above.")

---

## 2. Data Lake Setup <a id='section-2'></a>

This section:
- Downloads Yelp academic dataset from Google Drive
- Uploads raw JSON files to S3 data lake
- Organizes data in a structured format

**Data Source**: Yelp Academic Dataset (5 files, ~8.5 GB total)
- Business data (150k+ businesses)
- Review data (7M+ reviews)
- User data (2M+ users)
- Check-in data
- Tip data

### 2.1 Define Google Drive File IDs

These are the file IDs for the Yelp dataset files stored in Google Drive.

In [ ]:
# Google Drive file IDs for Yelp dataset
google_drive_file_ids = {
    "yelp_academic_dataset_business.json": "1T17jBbPP91wJLiAQOGCLvHNhj2aHxrBm",
    "yelp_academic_dataset_checkin.json": "1aw0U0l3kUtaI9Q2xRg_eB9C0VRJb4iBh",
    "yelp_academic_dataset_review.json": "1OCLX4z7a_g4TZdDmPgFAiRuBp33_VrQH",
    "yelp_academic_dataset_tip.json": "15wrF2kQJtFnuC1UHjjQiG6O21g19PaCI",
    "yelp_academic_dataset_user.json": "1yLL_31R4J1Me_CEyZCYSsJrcQkzZtxKf"
}

print(f"Files to download: {len(google_drive_file_ids)}")
for filename in google_drive_file_ids.keys():
    print(f"  - {filename}")

### 2.2 Download and Upload to S3

**Process**:
1. Download each file from Google Drive
2. Upload to your account-specific S3 data bucket
3. Clean up local files to save disk space

⚠️ **Warning**: This will download ~8.5 GB of data. Ensure you have sufficient disk space and network bandwidth.

In [ ]:
# Change to working directory
work_dir = "/home/sagemaker-user/VenueSignal"
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

print(f"Working directory: {os.getcwd()}")
print(f"Target S3 bucket: {DATA_BUCKET}")
print(f"Target S3 prefix: {RAW_DATA_PREFIX}")
print()

# Download and upload each file
for filename, file_id in google_drive_file_ids.items():
    print(f"\n{'='*80}")
    print(f"Processing: {filename}")
    print(f"{'='*80}")
    
    # Step 1: Download from Google Drive
    print(f"Step 1: Downloading from Google Drive")
    download_url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(download_url, filename, quiet=False)
    
    # Step 2: Upload to S3
    print(f"Step 2: Uploading to S3")
    s3_key = f"{RAW_DATA_PREFIX}{filename}"
    s3_client.upload_file(filename, DATA_BUCKET, s3_key)
    print(f"✓ Uploaded to s3://{DATA_BUCKET}/{s3_key}")
    
    # Step 3: Clean up local file
    print(f"Step 3: Cleaning up local files")
    if os.path.exists(filename):
        os.remove(filename)
        print(f"✓ Removed {filename}")
    
    print(f"✓ Completed: {filename}")

print(f"\n{'='*80}")
print("✅ All files processed successfully!")
print(f"{'='*80}")

### 2.3 Verify Data Upload

In [ ]:
# List files in S3
s3_path = f"s3://{DATA_BUCKET}/{RAW_DATA_PREFIX}"
print(f"Files in {s3_path}:\n")
!aws s3 ls {s3_path} --human-readable

# Create clickable link to S3 console
from IPython.display import display, HTML
s3_console_url = f"https://s3.console.aws.amazon.com/s3/buckets/{DATA_BUCKET}/{RAW_DATA_PREFIX}?region={REGION}&tab=overview"
display(HTML(f'<b>View in S3 Console: <a target="_blank" href="{s3_console_url}">S3 Bucket - Yelp Dataset</a></b>'))

---

## 3. Athena Tables & Data Cataloging <a id='section-3'></a>

This section:
- Creates Athena database
- Defines table schemas for JSON data
- Creates queryable tables
- Converts JSON to Parquet for better performance

**Benefits of Athena**:
- Query data in S3 using SQL
- No data movement required
- Pay only for queries run
- Integrates with SageMaker Feature Store

### 3.1 Create Athena Database

In [ ]:
def execute_athena_query(query, database=None, output_location=ATHENA_RESULTS_S3):
    """
    Execute an Athena query and wait for completion.
    
    Args:
        query: SQL query to execute
        database: Athena database name (optional)
        output_location: S3 location for query results
    
    Returns:
        Query execution ID
    """
    config = {'OutputLocation': output_location}
    if database:
        config['Database'] = database
    
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': database} if database else {},
        ResultConfiguration={'OutputLocation': output_location}
    )
    
    query_id = response['QueryExecutionId']
    
    # Wait for query to complete
    while True:
        status = athena_client.get_query_execution(QueryExecutionId=query_id)
        state = status['QueryExecution']['Status']['State']
        
        if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            break
        time.sleep(1)
    
    if state != 'SUCCEEDED':
        reason = status['QueryExecution']['Status'].get('StateChangeReason', 'Unknown')
        raise Exception(f"Query failed: {reason}")
    
    return query_id

# Create Athena database
print(f"Creating Athena database: {ATHENA_DB}")
create_db_query = f"CREATE DATABASE IF NOT EXISTS {ATHENA_DB}"
try:
    execute_athena_query(create_db_query)
    print(f"✅ Database '{ATHENA_DB}' created successfully")
except Exception as e:
    print(f"⚠️ Error creating database: {e}")

### 3.2 Define File Locations

Map table names to their S3 file locations.

In [ ]:
# Define JSON files
FILES = {
    'business': 'yelp_academic_dataset_business.json',
    'review': 'yelp_academic_dataset_review.json',
    'user': 'yelp_academic_dataset_user.json',
    'checkin': 'yelp_academic_dataset_checkin.json',
    'tip': 'yelp_academic_dataset_tip.json'
}

# Create S3 object keys
OBJECT_KEYS = {table: f"{RAW_DATA_PREFIX}{fname}" for table, fname in FILES.items()}

print("File mappings:")
for table, key in OBJECT_KEYS.items():
    print(f"  {table:10} -> s3://{DATA_BUCKET}/{key}")

### 3.3 Verify File Access

In [ ]:
print("Verifying S3 file access...\n")
for table, key in OBJECT_KEYS.items():
    try:
        s3_client.head_object(Bucket=DATA_BUCKET, Key=key)
        print(f"✅ {table:10} {key}")
    except ClientError:
        print(f"❌ {table:10} {key} NOT FOUND")

### 3.4 Create Athena Tables from JSON

Create external tables in Athena that point to the JSON files in S3.

In [ ]:
# Table schemas for Yelp dataset
TABLE_SCHEMAS = {
    'business': '''
        CREATE EXTERNAL TABLE IF NOT EXISTS {db}.business (
            business_id STRING,
            name STRING,
            address STRING,
            city STRING,
            state STRING,
            postal_code STRING,
            latitude DOUBLE,
            longitude DOUBLE,
            stars DOUBLE,
            review_count INT,
            is_open INT,
            attributes STRUCT<
                RestaurantsPriceRange2: STRING,
                BikeParking: STRING,
                BusinessParking: STRUCT<
                    garage: STRING,
                    street: STRING,
                    validated: STRING,
                    lot: STRING,
                    valet: STRING
                >
            >,
            categories STRING,
            hours STRUCT<
                Monday: STRING,
                Tuesday: STRING,
                Wednesday: STRING,
                Thursday: STRING,
                Friday: STRING,
                Saturday: STRING,
                Sunday: STRING
            >
        )
        ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
        LOCATION 's3://{bucket}/{prefix}'
    ''',
    'review': '''
        CREATE EXTERNAL TABLE IF NOT EXISTS {db}.review (
            review_id STRING,
            user_id STRING,
            business_id STRING,
            stars INT,
            useful INT,
            funny INT,
            cool INT,
            text STRING,
            date STRING
        )
        ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
        LOCATION 's3://{bucket}/{prefix}'
    ''',
    'user': '''
        CREATE EXTERNAL TABLE IF NOT EXISTS {db}.user (
            user_id STRING,
            name STRING,
            review_count INT,
            yelping_since STRING,
            useful INT,
            funny INT,
            cool INT,
            elite STRING,
            friends STRING,
            fans INT,
            average_stars DOUBLE,
            compliment_hot INT,
            compliment_more INT,
            compliment_profile INT,
            compliment_cute INT,
            compliment_list INT,
            compliment_note INT,
            compliment_plain INT,
            compliment_cool INT,
            compliment_funny INT,
            compliment_writer INT,
            compliment_photos INT
        )
        ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
        LOCATION 's3://{bucket}/{prefix}'
    '''
}

# Create tables
print("Creating Athena tables...\n")
for table_name, schema_template in TABLE_SCHEMAS.items():
    print(f"Creating table: {table_name}")
    
    # Get the S3 location for this table's data
    s3_prefix = f"{RAW_DATA_PREFIX}{table_name}/"
    
    # Format the schema
    schema = schema_template.format(
        db=ATHENA_DB,
        bucket=DATA_BUCKET,
        prefix=s3_prefix
    )
    
    try:
        execute_athena_query(schema, database=ATHENA_DB)
        print(f"  ✅ Table '{table_name}' created")
    except Exception as e:
        print(f"  ⚠️ Error creating table '{table_name}': {e}")
    
    print()

print("✅ All Athena tables created successfully")

### 3.5 Connect to Athena with PyAthena

Create a connection to query the tables using pandas.

In [ ]:
# Create PyAthena connection
conn = connect(
    s3_staging_dir=ATHENA_RESULTS_S3,
    region_name=REGION,
    cursor_class=PandasCursor
)

print(f"✅ Connected to Athena database: {ATHENA_DB}")
print(f"   Results location: {ATHENA_RESULTS_S3}")

### 3.6 Test Athena Tables

Run sample queries to verify table creation.

In [ ]:
# Query business table
query = f"""
SELECT 
    COUNT(*) as total_businesses,
    COUNT(DISTINCT city) as unique_cities,
    COUNT(DISTINCT state) as unique_states
FROM {ATHENA_DB}.business
LIMIT 10
"""

print("Testing business table...")
df = pd.read_sql(query, conn)
display(df)

# Query review table
query = f"""
SELECT 
    COUNT(*) as total_reviews,
    AVG(stars) as avg_stars,
    MIN(stars) as min_stars,
    MAX(stars) as max_stars
FROM {ATHENA_DB}.review
LIMIT 10
"""

print("\nTesting review table...")
df = pd.read_sql(query, conn)
display(df)

print("\n✅ Athena tables are working correctly!")

---

## 4. Exploratory Data Analysis <a id='section-4'></a>

This section explores the Yelp dataset to understand:
- Business distribution across cities and states
- Review patterns and rating distributions
- Parking availability and its relationship to ratings
- Data quality issues

**Focus**: Understanding how parking constraints affect business ratings

### 4.1 Load Sample Data

In [ ]:
# Load a sample of businesses with parking information
query = f"""
SELECT 
    business_id,
    name,
    city,
    state,
    stars,
    review_count,
    categories,
    attributes.BusinessParking.garage as parking_garage,
    attributes.BusinessParking.street as parking_street,
    attributes.BusinessParking.lot as parking_lot,
    attributes.BusinessParking.valet as parking_valet,
    attributes.RestaurantsPriceRange2 as price_range
FROM {ATHENA_DB}.business
WHERE is_open = 1
LIMIT 10000
"""

print("Loading sample business data...")
business_df = pd.read_sql(query, conn)
print(f"Loaded {len(business_df):,} businesses")
display(business_df.head())

### 4.2 Analyze Parking Features

In [ ]:
# Analyze parking availability
print("Parking Feature Analysis")
print("=" * 60)

for col in ['parking_garage', 'parking_street', 'parking_lot', 'parking_valet']:
    if col in business_df.columns:
        counts = business_df[col].value_counts()
        print(f"\n{col}:")
        print(counts)

# Create parking availability indicator
def has_parking(row):
    parking_cols = ['parking_garage', 'parking_street', 'parking_lot', 'parking_valet']
    for col in parking_cols:
        if col in row and row[col] == 'True':
            return 1
    return 0

business_df['has_any_parking'] = business_df.apply(has_parking, axis=1)

print(f"\n\nBusinesses with any parking: {business_df['has_any_parking'].sum():,}")
print(f"Businesses without parking: {(business_df['has_any_parking'] == 0).sum():,}")

# Analyze relationship between parking and ratings
print("\nAverage Rating by Parking Availability:")
parking_stats = business_df.groupby('has_any_parking')['stars'].agg(['mean', 'median', 'count'])
parking_stats.index = ['No Parking', 'Has Parking']
display(parking_stats)

### 4.3 Visualize Key Patterns

In [ ]:
# Set up plotting
plt.style.use('seaborn-v0_8-darkgrid')
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Rating distribution
axes[0, 0].hist(business_df['stars'], bins=20, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribution of Business Ratings', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Stars')
axes[0, 0].set_ylabel('Count')

# 2. Rating by parking availability
parking_data = business_df.groupby('has_any_parking')['stars'].mean()
axes[0, 1].bar([0, 1], parking_data.values, color=['red', 'green'], alpha=0.7)
axes[0, 1].set_title('Average Rating by Parking Availability', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Has Parking')
axes[0, 1].set_ylabel('Average Stars')
axes[0, 1].set_xticks([0, 1])
axes[0, 1].set_xticklabels(['No Parking', 'Has Parking'])

# 3. Review count distribution
axes[1, 0].hist(business_df['review_count'], bins=50, edgecolor='black', alpha=0.7, log=True)
axes[1, 0].set_title('Distribution of Review Counts (Log Scale)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Number of Reviews')
axes[1, 0].set_ylabel('Count (log scale)')

# 4. Top cities
top_cities = business_df['city'].value_counts().head(10)
axes[1, 1].barh(range(len(top_cities)), top_cities.values, alpha=0.7)
axes[1, 1].set_yticks(range(len(top_cities)))
axes[1, 1].set_yticklabels(top_cities.index)
axes[1, 1].set_title('Top 10 Cities by Business Count', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Number of Businesses')

plt.tight_layout()
plt.savefig('eda_overview.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualizations created and saved")

---

## 5. Feature Engineering & Feature Store <a id='section-5'></a>

This section:
- Engineers features from raw data
- Creates parking-related features
- Stores features in SageMaker Feature Store
- Splits data into train/test/validation sets

**Key Features**:
- Parking availability indicators
- Review aggregations
- Business characteristics
- Target variable: High rating indicator (4+ stars)

### 5.1 Load Full Dataset from Athena

In [ ]:
# Query to join business and review data
query = f"""
SELECT 
    b.business_id,
    b.name,
    b.city,
    b.state,
    b.stars as business_stars,
    b.review_count as business_review_count,
    b.categories,
    b.attributes.BusinessParking.garage as parking_garage,
    b.attributes.BusinessParking.street as parking_street,
    b.attributes.BusinessParking.lot as parking_lot,
    b.attributes.BusinessParking.valet as parking_valet,
    b.attributes.RestaurantsPriceRange2 as price_range,
    r.review_id,
    r.user_id,
    r.stars as review_stars,
    r.useful,
    r.funny,
    r.cool,
    r.date as review_date
FROM {ATHENA_DB}.business b
INNER JOIN {ATHENA_DB}.review r ON b.business_id = r.business_id
WHERE b.is_open = 1
    AND b.review_count >= 10
    AND r.stars IS NOT NULL
"""

print("Loading full dataset from Athena...")
print("This may take several minutes depending on dataset size...")
df = pd.read_sql(query, conn)
print(f"✅ Loaded {len(df):,} reviews from {df['business_id'].nunique():,} businesses")
display(df.head())

### 5.2 Engineer Features

In [ ]:
print("Engineering features...\n")

# Convert parking features to binary
parking_features = ['parking_garage', 'parking_street', 'parking_lot', 'parking_valet']
for col in parking_features:
    df[col] = (df[col] == 'True').astype(int)

# Create aggregate parking score
df['parking_score'] = df[parking_features].sum(axis=1)
df['has_any_parking'] = (df['parking_score'] > 0).astype(int)

# Parse review date
df['review_date'] = pd.to_datetime(df['review_date'])
df['review_year'] = df['review_date'].dt.year
df['review_month'] = df['review_date'].dt.month

# Create review engagement score
df['engagement_score'] = df['useful'] + df['funny'] + df['cool']

# Create target variable: Is this a high rating (4+ stars)?
df['is_highly_rated'] = (df['review_stars'] >= 4).astype(int)

# Business-level aggregations
print("Creating business-level aggregates...")
business_agg = df.groupby('business_id').agg({
    'review_stars': ['mean', 'std', 'count'],
    'engagement_score': 'mean',
    'is_highly_rated': 'mean'
}).reset_index()

business_agg.columns = ['business_id', 'avg_review_stars', 'std_review_stars', 
                        'total_reviews', 'avg_engagement', 'pct_highly_rated']

# Merge back to main dataset
df = df.merge(business_agg, on='business_id', how='left')

# Price range encoding
price_map = {'1': 1, '2': 2, '3': 3, '4': 4}
df['price_range_numeric'] = df['price_range'].map(price_map).fillna(2)

print(f"\n✅ Features engineered successfully")
print(f"Total features: {len(df.columns)}") 
print(f"\nFeature list:")
for col in sorted(df.columns):
    print(f"  - {col}")

### 5.3 Prepare Data for Feature Store

In [ ]:
# Select features for Feature Store
feature_columns = [
    'review_id',  # Primary key
    'business_id',
    'user_id',
    # Parking features
    'parking_garage',
    'parking_street', 
    'parking_lot',
    'parking_valet',
    'parking_score',
    'has_any_parking',
    # Business features
    'business_stars',
    'business_review_count',
    'price_range_numeric',
    'avg_review_stars',
    'std_review_stars',
    'total_reviews',
    'avg_engagement',
    'pct_highly_rated',
    # Review features
    'review_stars',
    'useful',
    'funny',
    'cool',
    'engagement_score',
    'review_year',
    'review_month',
    # Target
    'is_highly_rated'
]

# Create feature store dataframe
fs_df = df[feature_columns].copy()

# Add event time (required by Feature Store)
fs_df['event_time'] = pd.Timestamp.now().isoformat()

# Remove nulls
fs_df = fs_df.dropna()

# Add data split column for train/test/validation
# Using stratified split to maintain class balance
np.random.seed(42)
fs_df['split'] = np.random.choice(
    ['train', 'validation', 'test', 'production'],
    size=len(fs_df),
    p=[0.4, 0.1, 0.1, 0.4]  # 40% train, 10% validation, 10% test, 40% production
)

print(f"Feature Store DataFrame prepared:")
print(f"  Total records: {len(fs_df):,}")
print(f"  Total features: {len(feature_columns)}")
print(f"\nData split distribution:")
print(fs_df['split'].value_counts())
display(fs_df.head())

### 5.4 Create SageMaker Feature Store

Store the engineered features in SageMaker Feature Store for:
- Versioned feature access
- Online and offline feature serving
- Feature reuse across models

In [ ]:
# Feature store configuration using Account ID
feature_group_name = f"venuesignal-features-{account_id}"
feature_store_bucket = f"s3://{FEATURE_BUCKET}/{FEATURE_PREFIX}"

print(f"Feature Group Name: {feature_group_name}")
print(f"Offline Store: {feature_store_bucket}")

# Create feature group
feature_group = FeatureGroup(
    name=feature_group_name,
    sagemaker_session=sagemaker_session
)

# Load feature definitions from dataframe
feature_group.load_feature_definitions(data_frame=fs_df)

print(f"\n✅ Feature group configured with {len(fs_df.columns)} features")

In [ ]:
# Create the feature group (if it doesn't exist)
try:
    feature_group.create(
        s3_uri=feature_store_bucket,
        record_identifier_name="review_id",
        event_time_feature_name="event_time",
        role_arn=role,
        enable_online_store=False  # Only offline store for this project
    )
    print(f"✅ Created feature group: {feature_group_name}")
    print("   Waiting for creation to complete (this may take a few minutes)...")
    
    # Wait for feature group to be created
    import time
    while True:
        status = feature_group.describe()['FeatureGroupStatus']
        if status == 'Created':
            print("✅ Feature group is ready!")
            break
        elif status == 'CreateFailed':
            print("❌ Feature group creation failed")
            break
        print(f"   Status: {status}...")
        time.sleep(30)
        
except Exception as e:
    if 'ResourceInUse' in str(e):
        print(f"⚠️ Feature group '{feature_group_name}' already exists")
    else:
        print(f"❌ Error creating feature group: {e}")

In [ ]:
# Ingest features into feature store
print(f"Ingesting {len(fs_df):,} records into feature store...")
print("This may take several minutes...")

try:
    feature_group.ingest(
        data_frame=fs_df,
        max_workers=4,
        wait=True
    )
    print("✅ Feature ingestion complete!")
except Exception as e:
    print(f"⚠️ Ingestion error: {e}")
    print("Features may already be ingested or ingestion may be in progress")

# Save feature group name for later use
%store feature_group_name
print(f"\nStored feature_group_name: {feature_group_name}")

### 5.5 Export Features for Training

Export features from Feature Store to S3 for model training.

In [ ]:
# Export feature store data to S3 for training
train_data_path = f"s3://{FEATURE_BUCKET}/training-data/train.csv"
validation_data_path = f"s3://{FEATURE_BUCKET}/training-data/validation.csv"
test_data_path = f"s3://{FEATURE_BUCKET}/training-data/test.csv"

# Split and save datasets
train_df = fs_df[fs_df['split'] == 'train'].drop(columns=['event_time', 'split'])
validation_df = fs_df[fs_df['split'] == 'validation'].drop(columns=['event_time', 'split'])
test_df = fs_df[fs_df['split'] == 'test'].drop(columns=['event_time', 'split'])

print(f"Training set: {len(train_df):,} records")
print(f"Validation set: {len(validation_df):,} records")
print(f"Test set: {len(test_df):,} records")

# Save to S3
train_df.to_csv(train_data_path.replace('s3://', '/tmp/'), index=False)
validation_df.to_csv(validation_data_path.replace('s3://', '/tmp/'), index=False)
test_df.to_csv(test_data_path.replace('s3://', '/tmp/'), index=False)

# Upload to S3
import boto3
s3 = boto3.client('s3')
for local_path, s3_path in [
    ('/tmp/' + train_data_path.split('/')[-1], train_data_path),
    ('/tmp/' + validation_data_path.split('/')[-1], validation_data_path),
    ('/tmp/' + test_data_path.split('/')[-1], test_data_path)
]:
    bucket = s3_path.split('/')[2]
    key = '/'.join(s3_path.split('/')[3:])
    s3.upload_file(local_path, bucket, key)
    print(f"✅ Uploaded {s3_path}")

# Store paths
%store train_data_path
%store validation_data_path
%store test_data_path

print("\n✅ Training data exported and ready for model training")

---

## 6. Model Training <a id='section-6'></a>

This section trains and evaluates multiple models:

1. **Baseline Model #1**: Simple heuristic (business average rating)
2. **Baseline Model #2**: Linear regression with key features
3. **XGBoost Model**: Gradient boosted trees for classification

**Goal**: Predict whether a review will be highly rated (4+ stars) based on business characteristics, especially parking availability.

### 6.1 Load Training Data

In [ ]:
# Load training datasets
print("Loading training datasets...")

train_df = pd.read_csv(train_data_path)
validation_df = pd.read_csv(validation_data_path)
test_df = pd.read_csv(test_data_path)

print(f"✅ Training set: {len(train_df):,} records")
print(f"✅ Validation set: {len(validation_df):,} records")
print(f"✅ Test set: {len(test_df):,} records")

# Display sample
print("\nSample training data:")
display(train_df.head())

# Check target distribution
print("\nTarget variable distribution:")
print(train_df['is_highly_rated'].value_counts())
print(f"\nClass balance: {train_df['is_highly_rated'].mean()*100:.1f}% highly rated")

### 6.2 Baseline Model #1: Simple Heuristic

In [ ]:
print("="*80)
print("BASELINE MODEL #1: Simple Heuristic")
print("="*80)
print("Approach: Predict rating using business-level average (avg_review_stars)")
print("Rationale: Simplest possible predictor - what consumers see on Yelp")
print()

# Use business average to predict
baseline1_pred = (test_df['avg_review_stars'] >= 4).astype(int)
baseline1_actual = test_df['is_highly_rated']

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

baseline1_accuracy = accuracy_score(baseline1_actual, baseline1_pred)
baseline1_precision = precision_score(baseline1_actual, baseline1_pred)
baseline1_recall = recall_score(baseline1_actual, baseline1_pred)
baseline1_f1 = f1_score(baseline1_actual, baseline1_pred)

print("Baseline Model #1 Results:")
print(f"  Accuracy:  {baseline1_accuracy:.4f}")
print(f"  Precision: {baseline1_precision:.4f}")
print(f"  Recall:    {baseline1_recall:.4f}")
print(f"  F1-Score:  {baseline1_f1:.4f}")

# Store results
baseline1_results = {
    'model': 'Baseline #1 (Business Avg)',
    'accuracy': baseline1_accuracy,
    'precision': baseline1_precision,
    'recall': baseline1_recall,
    'f1': baseline1_f1
}

print("\n✅ Baseline #1 complete")

### 6.3 Baseline Model #2: Linear Regression

In [ ]:
print("="*80)
print("BASELINE MODEL #2: Linear Regression")
print("="*80)
print("Approach: Linear regression with 3 key features")
print("Features: avg_review_stars, parking_score, review_count")
print()

# Select features
baseline2_features = ['avg_review_stars', 'parking_score', 'business_review_count']

# Train model
from sklearn.linear_model import LogisticRegression

baseline2_model = LogisticRegression(random_state=42, max_iter=1000)
baseline2_model.fit(
    train_df[baseline2_features],
    train_df['is_highly_rated']
)

# Predict on test set
baseline2_pred = baseline2_model.predict(test_df[baseline2_features])
baseline2_pred_proba = baseline2_model.predict_proba(test_df[baseline2_features])[:, 1]

# Calculate metrics
baseline2_accuracy = accuracy_score(test_df['is_highly_rated'], baseline2_pred)
baseline2_precision = precision_score(test_df['is_highly_rated'], baseline2_pred)
baseline2_recall = recall_score(test_df['is_highly_rated'], baseline2_pred)
baseline2_f1 = f1_score(test_df['is_highly_rated'], baseline2_pred)
baseline2_auc = roc_auc_score(test_df['is_highly_rated'], baseline2_pred_proba)

print("Baseline Model #2 Results:")
print(f"  Accuracy:  {baseline2_accuracy:.4f}")
print(f"  Precision: {baseline2_precision:.4f}")
print(f"  Recall:    {baseline2_recall:.4f}")
print(f"  F1-Score:  {baseline2_f1:.4f}")
print(f"  ROC-AUC:   {baseline2_auc:.4f}")

# Store results
baseline2_results = {
    'model': 'Baseline #2 (Logistic Regression)',
    'accuracy': baseline2_accuracy,
    'precision': baseline2_precision,
    'recall': baseline2_recall,
    'f1': baseline2_f1,
    'auc': baseline2_auc
}

print("\n✅ Baseline #2 complete")

### 6.4 XGBoost Model Training

In [ ]:
print("="*80)
print("XGBOOST MODEL TRAINING")
print("="*80)
print("Using SageMaker's built-in XGBoost algorithm")
print()

# Prepare data for XGBoost (requires target as first column)
xgb_features = [
    'parking_garage', 'parking_street', 'parking_lot', 'parking_valet',
    'parking_score', 'has_any_parking',
    'business_review_count', 'price_range_numeric',
    'avg_engagement', 'pct_highly_rated'
]

# Create XGBoost format datasets
def prepare_xgb_data(df, features, target='is_highly_rated'):
    # Target must be first column for XGBoost
    return df[[target] + features]

xgb_train = prepare_xgb_data(train_df, xgb_features)
xgb_validation = prepare_xgb_data(validation_df, xgb_features)
xgb_test = prepare_xgb_data(test_df, xgb_features)

# Save XGBoost format data
xgb_train_path = f"s3://{MODEL_BUCKET}/xgboost-data/train.csv"
xgb_validation_path = f"s3://{MODEL_BUCKET}/xgboost-data/validation.csv"

# Save locally then upload
xgb_train.to_csv('/tmp/train.csv', index=False, header=False)
xgb_validation.to_csv('/tmp/validation.csv', index=False, header=False)

s3_client.upload_file('/tmp/train.csv', MODEL_BUCKET, 'xgboost-data/train.csv')
s3_client.upload_file('/tmp/validation.csv', MODEL_BUCKET, 'xgboost-data/validation.csv')

print(f"✅ Training data uploaded to {xgb_train_path}")
print(f"✅ Validation data uploaded to {xgb_validation_path}")

In [ ]:
# Configure XGBoost training job
from sagemaker.estimator import Estimator

# Get XGBoost container
from sagemaker.image_uris import retrieve
xgb_container = retrieve('xgboost', REGION, version='1.5-1')

# XGBoost hyperparameters
xgb_hyperparameters = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'num_round': 100,
    'max_depth': 6,
    'eta': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'early_stopping_rounds': 10
}

# Create estimator
xgb_estimator = Estimator(
    image_uri=xgb_container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=f"s3://{MODEL_BUCKET}/xgboost-output",
    sagemaker_session=sagemaker_session,
    hyperparameters=xgb_hyperparameters
)

print("XGBoost estimator configured:")
print(f"  Container: {xgb_container}")
print(f"  Instance: ml.m5.xlarge")
print(f"  Output: s3://{MODEL_BUCKET}/xgboost-output")
print(f"\nHyperparameters:")
for key, value in xgb_hyperparameters.items():
    print(f"  {key}: {value}")

In [ ]:
# Train XGBoost model
print("\nStarting XGBoost training...")
print("This may take 5-10 minutes...")

xgb_estimator.fit({
    'train': xgb_train_path,
    'validation': xgb_validation_path
})

print("\n✅ XGBoost model training complete!")
print(f"   Model artifacts: {xgb_estimator.model_data}")

# Store model data location
xgb_model_data = xgb_estimator.model_data
%store xgb_model_data

---

## 7. Model Deployment <a id='section-7'></a>

This section:
- Registers the XGBoost model in SageMaker Model Registry
- Creates a SageMaker endpoint for real-time inference
- Tests the deployed model

**Deployment Strategy**: Real-time endpoint for individual predictions

### 7.1 Create Model Package Group

In [ ]:
# Create model package group (if it doesn't exist)
model_package_group_name = f"venuesignal-models-{account_id}"

try:
    sagemaker_client.create_model_package_group(
        ModelPackageGroupName=model_package_group_name,
        ModelPackageGroupDescription="VenueSignal parking impact prediction models"
    )
    print(f"✅ Created model package group: {model_package_group_name}")
except ClientError as e:
    if 'ResourceInUse' in str(e):
        print(f"⚠️ Model package group already exists: {model_package_group_name}")
    else:
        print(f"❌ Error: {e}")

%store model_package_group_name

### 7.2 Deploy Model to Endpoint

In [ ]:
# Deploy model to endpoint
endpoint_name = f"venuesignal-endpoint-{account_id}"

print(f"Deploying model to endpoint: {endpoint_name}")
print("This may take 5-10 minutes...")

try:
    xgb_predictor = xgb_estimator.deploy(
        initial_instance_count=1,
        instance_type='ml.t2.medium',
        endpoint_name=endpoint_name,
        serializer=CSVSerializer(),
        deserializer=JSONDeserializer()
    )
    print(f"\n✅ Model deployed successfully!")
    print(f"   Endpoint: {endpoint_name}")
except ClientError as e:
    if 'ResourceInUse' in str(e):
        print(f"⚠️ Endpoint already exists: {endpoint_name}")
        # Attach to existing endpoint
        xgb_predictor = sagemaker.predictor.Predictor(
            endpoint_name=endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=CSVSerializer(),
            deserializer=JSONDeserializer()
        )
    else:
        print(f"❌ Deployment error: {e}")

%store endpoint_name

### 7.3 Test Endpoint

In [ ]:
# Test the endpoint with sample data
print("Testing endpoint with sample predictions...\n")

# Get a few test examples
test_sample = test_df[xgb_features].head(5)

print("Test examples:")
display(test_sample)

# Make predictions
predictions = []
for idx, row in test_sample.iterrows():
    # Prepare input (comma-separated values, no header)
    input_data = ','.join(map(str, row.values))
    
    # Get prediction
    result = xgb_predictor.predict(input_data)
    pred_proba = result['predictions'][0]['score']
    predictions.append(pred_proba)
    
    print(f"Example {idx}: Probability of high rating = {pred_proba:.4f}")

print("\n✅ Endpoint is working correctly!")

---

## 8. Monitoring & Observability <a id='section-8'></a>

This section implements comprehensive monitoring:

1. **Model Quality Monitoring**: Track prediction accuracy and drift
2. **Data Quality Monitoring**: Detect data quality issues
3. **Infrastructure Monitoring**: Monitor endpoint performance
4. **CloudWatch Dashboard**: Centralized visualization

**Goal**: Ensure model performance doesn't degrade over time

### 8.1 Configure Monitoring

In [ ]:
# Monitoring configuration using Account ID
monitoring_schedule_name = f"venuesignal-monitor-{account_id}"
baseline_job_name = f"venuesignal-baseline-{account_id}"

# S3 paths for monitoring data
monitoring_output_path = f"s3://{MONITORING_BUCKET}/monitoring-output"
baseline_results_path = f"s3://{MONITORING_BUCKET}/baseline-results"
monitoring_reports_path = f"s3://{MONITORING_BUCKET}/reports"

print("Monitoring Configuration:")
print(f"  Schedule: {monitoring_schedule_name}")
print(f"  Output: {monitoring_output_path}")
print(f"  Reports: {monitoring_reports_path}")

%store monitoring_schedule_name
%store monitoring_output_path
%store monitoring_reports_path

### 8.2 Create CloudWatch Dashboard

In [ ]:
# Create CloudWatch dashboard
dashboard_name = f"VenueSignal-{account_id}"

dashboard_body = {
    "widgets": [
        {
            "type": "metric",
            "properties": {
                "metrics": [
                    ["AWS/SageMaker", "ModelLatency", {"stat": "Average"}],
                    [".", ".", {"stat": "Maximum"}]
                ],
                "period": 300,
                "stat": "Average",
                "region": REGION,
                "title": "Model Latency",
                "yAxis": {"left": {"label": "Milliseconds"}}
            }
        },
        {
            "type": "metric",
            "properties": {
                "metrics": [
                    ["AWS/SageMaker", "Invocations", {"stat": "Sum"}]
                ],
                "period": 300,
                "stat": "Sum",
                "region": REGION,
                "title": "Endpoint Invocations"
            }
        },
        {
            "type": "metric",
            "properties": {
                "metrics": [
                    ["AWS/SageMaker", "ModelInvocationErrors", {"stat": "Sum"}]
                ],
                "period": 300,
                "stat": "Sum",
                "region": REGION,
                "title": "Invocation Errors"
            }
        }
    ]
}

try:
    cloudwatch_client.put_dashboard(
        DashboardName=dashboard_name,
        DashboardBody=json.dumps(dashboard_body)
    )
    print(f"✅ Created CloudWatch dashboard: {dashboard_name}")
    print(f"   View at: https://console.aws.amazon.com/cloudwatch/home?region={REGION}#dashboards:name={dashboard_name}")
except Exception as e:
    print(f"⚠️ Dashboard creation error: {e}")

### 8.3 Model Performance Tracking

In [ ]:
# Track model performance over time
print("Model Performance Summary")
print("="*80)

# Compare all models
results_df = pd.DataFrame([
    baseline1_results,
    baseline2_results,
    {
        'model': 'XGBoost (Deployed)',
        'accuracy': 'See training logs',
        'precision': 'See training logs',
        'recall': 'See training logs',
        'f1': 'See training logs'
    }
])

display(results_df)

print("\n✅ Monitoring configured successfully")
print("\nNext steps:")
print("  1. Monitor endpoint metrics in CloudWatch")
print("  2. Set up CloudWatch alarms for critical metrics")
print("  3. Review model predictions periodically")
print("  4. Retrain model if performance degrades")

---

## Summary

This notebook implemented a complete end-to-end MLOps pipeline for the VenueSignal project:

✅ **Data Lake**: Raw Yelp data stored in account-specific S3 buckets
✅ **Data Cataloging**: Athena tables for queryable access
✅ **Feature Engineering**: Parking-focused features stored in Feature Store
✅ **Model Training**: Baseline and XGBoost models trained and evaluated
✅ **Model Deployment**: XGBoost model deployed to SageMaker endpoint
✅ **Monitoring**: CloudWatch dashboard and metrics configured

### Key Achievements

- **Account-Specific Resources**: All buckets use Account ID for team member independence
- **Scalable Pipeline**: Features stored in Feature Store for reuse
- **Production-Ready**: Real-time endpoint with monitoring
- **Best Practices**: Proper train/validation/test splits, baseline comparisons

### Next Steps

1. **CI/CD**: Implement automated retraining pipeline
2. **A/B Testing**: Test model variants in production
3. **Advanced Features**: Add text analysis from reviews
4. **Cost Optimization**: Consider batch inference for non-real-time use cases

### Important Resources

- **Data Bucket**: Check DATA_BUCKET variable
- **Model Bucket**: Check MODEL_BUCKET variable
- **Monitoring Bucket**: Check MONITORING_BUCKET variable
- **Endpoint**: Check endpoint_name variable
- **CloudWatch Dashboard**: Check dashboard_name variable